In [1]:
# !pip install -q evaluate
# !pip install -q rouge_score

## Import Libraries

In [2]:
import nltk
import evaluate
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset,DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,T5Tokenizer,T5ForConditionalGeneration
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer,  DataCollatorForTokenClassification
from tqdm import tqdm
import evaluate
import os
os.environ["WANDB_DISABLED"] = "true"

## Load flan-t5 model

In [3]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "/root/autodl-tmp/models/codet5-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Split dataset for training

In [4]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-annotated/dataset.csv', index_col=0, encoding='utf_8_sig')
df

,maintenance_type,commit_diff,commit_message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_DEL_LINE,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE
commit_sha,,,,,,,,,,,,,,,,,,,,,
be937a3290223f926fe50684f1344569a573ed4b,C,diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
c800d2e36954edddcb83aa1df7f623f2780c7780,A,diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
469e56eeff17eb857c95e935ca7b49723c43470e,P,diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...,0,0,0,0,0,0,0,...,0,0,8,0,0,15,0,0,0,0
24f5d302c0906f3377f4d6bcc2735d943ce13e3d,A,diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,...",3,37,0,0,9,0,0,...,0,0,0,0,0,0,1,2,0,0
1effdb7b959503596b07f7f3e74618ab63f9e5e5,P,diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...,1,0,12,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,C,diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,C,diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...,2,6,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,C,diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...,1,1,5,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:

df.fillna('', inplace=True)
# label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'labels','commit_diff':'diffs'})
# df = df.replace({"labels": label2id})
# # print(df)
df['text'] = df['commit_message'].astype(str) + df['diffs'].astype(str)
df
# test_sample = df.sample(n=3, random_state=1)

,labels,diffs,commit_message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE,text
commit_sha,,,,,,,,,,,,,,,,,,,,,
be937a3290223f926fe50684f1344569a573ed4b,C,diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,Fixing per axis quantization bug in flatbuffer...
c800d2e36954edddcb83aa1df7f623f2780c7780,A,diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Add support for python expression as SavedMode...
469e56eeff17eb857c95e935ca7b49723c43470e,P,diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...,0,0,0,0,0,0,0,...,0,8,0,0,15,0,0,0,0,Fix GitHub issue templates. Actual fix for #36...
24f5d302c0906f3377f4d6bcc2735d943ce13e3d,A,diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,...",3,37,0,0,9,0,0,...,0,0,0,0,0,1,2,0,0,"Add new PjRT distributed APIs (KeyValueDelete,..."
1effdb7b959503596b07f7f3e74618ab63f9e5e5,P,diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...,1,0,12,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,C,diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,fixing reflection constructor to use int inste...
75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,C,diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...,2,6,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,Fix OutOfBoundException on- MemoryFileSystem.e...
6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,C,diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...,1,1,5,13,0,0,0,...,0,0,0,0,0,0,0,0,0,[DROOLS-293] fix ObjectTypeNode id creation an...


In [6]:
full_dataset = Dataset.from_pandas(df)

# 1) train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
first_split = full_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = first_split["train"]      # ~70%
test_tmp  = first_split["test"]       # ~30%

# 2) test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
second_split = test_tmp.train_test_split(test_size=0.5, seed=42)
test_dataset  = second_split["train"]     # ~15%
valid_dataset   = second_split["test"]      # ~15%  （后面会被覆盖，和你 pandas 逻辑一致）




label_col = "labels"  # ← 改成你的 label 列名

train_70_df = train_dataset.to_pandas()

# 每类抽 5 个
train_5_df = (
    train_70_df.groupby(label_col, group_keys=False)
               .apply(lambda x: x.sample(n=10, random_state=42))
               .reset_index(drop=True)
)

# 转回 HF dataset
train_5_dataset = Dataset.from_pandas(train_5_df)

ds_splits = DatasetDict({
    "train": train_5_dataset,
    "valid": valid_dataset,
    "test":  test_dataset,
})

/tmp/ipykernel_3680/4240748031.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=10, random_state=42))


In [7]:
# train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
# test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
# train_data,_ = train_test_split(train_data, train_size=0.1, random_state=42)
# test_data, val_data = train_test_split(test_data, train_size=0.1, random_state=42)
# val_data, _ = train_test_split(val_data, train_size=0.1, random_state=42)

In [8]:
print("Before:\n", full_dataset)
print("After\n", ds_splits)

Before:
 Dataset({
    features: ['labels', 'diffs', 'commit_message', 'CODE_CHANGE_FILE', 'CODE_ADD_CODE_LINE', 'CODE_DEL_CODE_LINE', 'CODE_MOD_CODE_LINE', 'CODE_ADD_COMMENT_LINE', 'CODE_DEL_COMMENT_LINE', 'CODE_MOD_COMMENT_LINE', 'TEST_CHANGE_FILE', 'TEST_ADD_CODE_LINE', 'TEST_DEL_CODE_LINE', 'TEST_MOD_CODE_LINE', 'TEST_ADD_COMMENT_LINE', 'TEST_DEL_COMMENT_LINE', 'TEST_MOD_COMMENT_LINE', 'TEST_ADD_OTHER_LINE', 'TEST_DEL_OTHER_LINE', 'TEST_MOD_OTHER_LINE', 'BUILD_CHANGE_FILE', 'BUILD_ADD_LINE', 'BUILD_DEL_LINE', 'BUILD_MOD_LINE', 'DOC_CHANGE_FILE', 'DOC_ADD_LINE', 'DOC_DEL_LINE', 'DOC_MOD_LINE', 'OTHER_CHANGE_FILE', 'OTHER_ADD_LINE', 'OTHER_DEL_LINE', 'OTHER_MOD_LINE', 'text', 'commit_sha'],
    num_rows: 1581
})
After
 DatasetDict({
    train: Dataset({
        features: ['labels', 'diffs', 'commit_message', 'CODE_CHANGE_FILE', 'CODE_ADD_CODE_LINE', 'CODE_DEL_CODE_LINE', 'CODE_MOD_CODE_LINE', 'CODE_ADD_COMMENT_LINE', 'CODE_DEL_COMMENT_LINE', 'CODE_MOD_COMMENT_LINE', 'TEST_CHANGE_FILE

## Post-processing

In [9]:
# int_to_str = {0:'0', 1:'1'}

# prefix = "Please check whether the tweet is about a real disaster or not: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = examples["text"]
   model_inputs = tokenizer(inputs, max_length=128, padding=True, truncation=True)
    
   labels = tokenizer(text_target=examples["labels"], padding=True, truncation=True)
   model_inputs["labels"] = labels["input_ids"]

   return model_inputs

In [10]:
# Map the preprocessing function across our dataset
tokenized_dataset = ds_splits.map(preprocess_function, batched=False, remove_columns=['text','labels'])

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

Map:   0%|          | 0/237 [00:00<?, ? examples/s]

## Evaluation using ROGUE Score

In [11]:
# nltk.download("punkt", quiet=True)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

str_to_int = {'A':0, 'C':1, 'P':2}

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # 替换 label 中的 -100 为 pad，避免 decode 出错
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # 解码成字符串
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 去掉首尾空格
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    # string → int (positive / negative)
    y_pred = [str_to_int.get(p, -1) for p in decoded_preds]
    y_true = [str_to_int.get(l, -1) for l in decoded_labels]

    # 计算四个指标
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


## Training

In [12]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 32
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 10

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   eval_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   logging_steps = 100
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

/tmp/ipykernel_3680/2420821063.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,5.150611,0.000000,0.000000,0.000000,0.000000
2,No log,1.212211,0.000000,0.000000,0.000000,0.000000
3,No log,0.528176,0.291139,0.472403,0.454708,0.284783
4,No log,0.477637,0.628692,0.209564,0.333333,0.257340
5,No log,0.661835,0.160338,0.163059,0.346154,0.112876
6,No log,0.519197,0.219409,0.129149,0.336182,0.136014
7,No log,0.504379,0.219409,0.073446,0.333333,0.120370
8,No log,0.320103,0.628692,0.471259,0.469923,0.467756
9,No log,0.392974,0.400844,0.495799,0.451079,0.338472
10,No log,0.409374,0.350211,0.524796,0.453646,0.320566


/root/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/root/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/root/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/root/miniconda3/lib/python

TrainOutput(global_step=10, training_loss=1.9523958206176757, metrics={'train_runtime': 19.8786, 'train_samples_per_second': 15.092, 'train_steps_per_second': 0.503, 'total_flos': 45671841792000.0, 'train_loss': 1.9523958206176757, 'epoch': 10.0})

In [15]:
fewshot_metrics = trainer.evaluate(eval_dataset=tokenized_dataset["valid"])
fewshot_metrics

{'eval_loss': 0.4101266860961914,
 'eval_accuracy': 0.35294117647058826,
 'eval_precision': 0.5064935064935066,
 'eval_recall': 0.41818181818181815,
 'eval_f1': 0.327598232969077,
 'eval_runtime': 1.5303,
 'eval_samples_per_second': 155.528,
 'eval_steps_per_second': 19.604,
 'epoch': 10.0}